1. Load the paths

In [16]:
import os
import numpy as np

paths = [
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Amin\amin_eeg_pp.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Amin1\amin-1_eeg_pp.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Ismayil\ismyil_eeg_pp.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Marjan\marjan_eeg_pp.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Mina\mina_eeg_pp.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Mina 1\mina-1_eeg_pp.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Mina 3\mina-3_eeg_pp.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Mohammad\mohommad_eeg_pp.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Mona\mona_eeg_pp.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Cole\cole_eeg_pp.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Daniel\daniel_eeg_pp.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Jack\jack_eeg_pp.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\James\james_eeg_pp.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Josh\josh_eeg_pp.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Max\max_eeg_pp.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Roddy\roddy_eeg_pp.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Sam\sam_eeg_pp.npy",
    r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\adam\adam_eeg_pp.npy",
]

def head_along_first_axis(arr, n=5):
    if arr.ndim == 0:
        return arr  # scalar
    idx = (slice(0, min(n, arr.shape[0])),) + (slice(None),) * (arr.ndim - 1)
    return arr[idx]

for p in paths:
    print("\n" + "-"*88)
    print(p)
    try:
        # allow_pickle=True in case labels are object arrays (e.g., strings)
        arr = np.load(p, allow_pickle=True)
        print(f"Loaded: shape={arr.shape}, dtype={arr.dtype}")
        preview = head_along_first_axis(arr, n=5)
        print("First 5 lines (along axis 0):")
        print(preview)
    except FileNotFoundError:
        print("File not found.")
    except Exception as e:
        print(f"Error reading file: {e}")


----------------------------------------------------------------------------------------
C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Amin\amin_eeg_pp.npy
Loaded: shape=(265, 14, 200), dtype=float64
First 5 lines (along axis 0):
[[[ 5.90377385e-01  4.46700599e-01  1.79644750e+00 ... -4.06415843e+00
   -1.27143094e+00  4.17048461e+00]
  [ 5.25406818e+00  1.31528313e+00  4.46884709e-01 ... -2.52297349e+00
   -5.39772475e+00  1.31036090e+00]
  [ 2.09596518e+00 -5.86638881e-01 -1.73895392e-01 ... -7.09363669e-01
   -5.06137832e-01  5.06690725e-01]
  ...
  [-7.59439014e+00 -2.64269161e+00  6.82335478e-01 ... -3.57352232e+00
    7.63677293e+00  2.84898431e+00]
  [ 3.23791315e-01 -7.99008051e-01  1.58915110e+00 ... -4.90308888e+00
    9.67945625e+00  7.98891406e+00]
  [-3.13627692e+00 -6.78902282e+00 -3.30151176e+00 ... -2.40156535e+00
    5.37696819e+00  1.32954814e+00]]

 [[ 5.78573354e+00  3.27530982e+00  1.36681127e+00 ... -1.08485079e+00
   

2. Conacatenate arrays

In [17]:
expected = (265, 14, 200)

def load_eeg(path):
    """
    Load a .npy that may have been saved as an object array.
    Converts to a proper numeric ndarray with shape (265,14,200).
    """
    arr = np.load(path, allow_pickle=True)  # needed because source is object array
    if arr.dtype == object:
        # Try to stack element-wise if it's a 1D object array of (14,200) slices
        try:
            arr = np.stack([np.asarray(x) for x in arr], axis=0)
        except Exception:
            # Fall back to best-effort coercion
            arr = np.asarray(arr)
    if arr.shape != expected:
        raise ValueError(f"{os.path.basename(path)} has shape {arr.shape}, expected {expected}")
    # Ensure a consistent dtype (float32 is common for EEG; keep original if you want)
    if not np.issubdtype(arr.dtype, np.number):
        arr = arr.astype(np.float32, copy=False)
    return arr

# Load, validate, and concatenate
chunks = [load_eeg(p) for p in paths]
final_arr = np.concatenate(chunks, axis=0)


In [18]:
# Final sanity check: (4770, 14, 200)
assert final_arr.shape == (len(paths) * expected[0], expected[1], expected[2])
print("Final shape:", final_arr.shape, "dtype:", final_arr.dtype)

Final shape: (4770, 14, 200) dtype: float64


3. Save the final array

In [19]:
# Save alongside the first file
output_path = os.path.join(os.path.dirname(paths[0]), "final_eeg_pp.npy")
np.save(output_path, final_arr)
print("Saved to:", output_path)

Saved to: C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Amin\final_eeg_pp.npy


In [20]:
import os
import shutil

# Use the output_path variable from earlier if it exists; otherwise fall back to the Amin folder
try:
    src = output_path  # from your previous run
except NameError:
    src = r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Amin\final_eeg.npy"

dst_dir = r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\final eeg data"
os.makedirs(dst_dir, exist_ok=True)
dst = os.path.join(dst_dir, "final_eeg_pp.npy")

# Overwrite if file already exists
if os.path.exists(dst):
    os.remove(dst)

shutil.move(src, dst)  # use shutil.copy2(src, dst) if you prefer to keep the original
print(f"Final array moved to: {dst}")


Final array moved to: C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\final eeg data\final_eeg_pp.npy
